# (E6) 프로젝트: 멋진 작사가 만들기

## 1. 데이터 다운로드

In [66]:
# ! wget https://aiffelstaticprd.blob.core.windows.net/media/documents/song_lyrics.zip
# ! unzip song_lyrics.zip -d ~/aiffel/lyricist/data/lyrics  #lyrics 폴더에 압축풀기

## 2. 데이터 읽어오기

glob 모듈을 사용하면 파일을 읽어오는 작업을 하기가 아주 용이해요. glob 를 활용하여 모든 txt 파일을 읽어온 후, raw_corpus 리스트에 문장 단위로 저장하도록 할게요!

In [1]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:10])

데이터 크기: 187088
Examples:
 ['The first words that come out', 'And I can see this song will be about you', "I can't believe that I can breathe without you", 'But all I need to do is carry on', 'The next line I write down', "And there's a tear that falls between the pages", "I know that pain's supposed to heal in stages", "But it depends which one I'm standing on I write lines down, then rip them up", "Describing love can't be this tough I could set this song on fire, send it up in smoke", 'I could throw it in the river and watch it sink in slowly']


## 3. 데이터 정제

- preprocess_sentence() 함수를 이용해 데이터를 정제하겠습니다. 
- 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거하겠습니다.
- 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습데이터에서 제외합니다.

In [2]:
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf    # 대망의 텐서플로우!



def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


In [3]:
corpus = []

for sentence in :
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> the first words that come out <end>',
 '<start> and i can see this song will be about you <end>',
 '<start> i can t believe that i can breathe without you <end>',
 '<start> but all i need to do is carry on <end>',
 '<start> the next line i write down <end>',
 '<start> and there s a tear that falls between the pages <end>',
 '<start> i know that pain s supposed to heal in stages <end>',
 '<start> but it depends which one i m standing on i write lines down , then rip them up <end>',
 '<start> describing love can t be this tough i could set this song on fire , send it up in smoke <end>',
 '<start> i could throw it in the river and watch it sink in slowly <end>']

In [4]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2   6 241 ...   0   0   0]
 [  2   8   5 ...   0   0   0]
 [  2   5  32 ...   0   0   0]
 ...
 [  2 133 117 ...   0   0   0]
 [  2 133 117 ...   0   0   0]
 [  2 133 117 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f4a54ca5e10>


In [5]:
print(tensor[:3, :15])

[[  2   6 241 426  17  66  56   3   0   0   0   0   0   0   0]
 [  2   8   5  32  64  42 334  88  27 113   7   3   0   0   0]
 [  2   5  32  15 217  17   5  32 795 258   7   3   0   0   0]]


In [6]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [7]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[  2   6 241 426  17  66  56   3   0   0   0   0   0   0]
[  6 241 426  17  66  56   3   0   0   0   0   0   0   0]


## 4. 평가 데이터셋 분리

- 훈련 데이터와 평가 데이터를 분리합니다.
- 총 데이터의 20%를 평가 데이터셋으로 사용합니다.

In [8]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=1)

In [9]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [10]:
dataset_val = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)).shuffle(BUFFER_SIZE)
dataset_val = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset_val

<BatchDataset shapes: ((256, 256, 14), (256, 256, 14)), types: (tf.int32, tf.int32)>

In [11]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140599, 14)
Target Train: (140599, 14)


만약 결과가 다르다면 천천히 과정을 다시 살펴 동일한 결과를 얻도록 하세요! 만약 학습데이터 갯수가 124960보다 크다면 위 Step 3.의 데이터 정제 과정을 다시한번 검토해 보시기를 권합니다.

$- >$ 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습데이터에서 제외하였습니다.

## 5. 인공지능 만들기

In [12]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [13]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-8.93310789e-05,  1.78819944e-04,  1.74000510e-04, ...,
         -1.99288726e-04,  9.79945544e-05, -1.56782320e-04],
        [-3.13257711e-04,  1.73907727e-04,  4.54257999e-04, ...,
          1.47738774e-05,  1.62724740e-04, -3.78803205e-04],
        [-3.23566812e-04,  1.24466489e-04,  4.83063282e-04, ...,
          3.00054147e-04,  3.03616671e-05, -4.53804940e-04],
        ...,
        [ 1.21042109e-03,  1.62642729e-03, -1.17626099e-03, ...,
          1.84738229e-03,  1.13646861e-03, -1.62270817e-03],
        [ 1.52494886e-03,  1.60777429e-03, -1.63346110e-03, ...,
          1.96431088e-03,  1.21626898e-03, -1.92542805e-03],
        [ 1.82063552e-03,  1.52868393e-03, -2.07377481e-03, ...,
          2.06462410e-03,  1.24735176e-03, -2.17479514e-03]],

       [[-8.93310789e-05,  1.78819944e-04,  1.74000510e-04, ...,
         -1.99288726e-04,  9.79945544e-05, -1.56782320e-04],
        [-2.03646603e-04,  4.19981778e-04,  4

In [14]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


## 모델 학습

In [20]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10, validation_data=(enc_val, dec_val), verbose=1)

Epoch 1/10
549/549 [==============================] - 77s 140ms/step - loss: 2.7997 - val_loss: 2.8368
Epoch 2/10
549/549 [==============================] - 79s 144ms/step - loss: 2.6013 - val_loss: 2.7593
Epoch 3/10
549/549 [==============================] - 81s 147ms/step - loss: 2.4360 - val_loss: 2.7042
Epoch 4/10
549/549 [==============================] - 77s 139ms/step - loss: 2.2899 - val_loss: 2.6613
Epoch 5/10
549/549 [==============================] - 77s 139ms/step - loss: 2.1588 - val_loss: 2.6237
Epoch 6/10
549/549 [==============================] - 77s 139ms/step - loss: 2.0388 - val_loss: 2.5984
Epoch 7/10
549/549 [==============================] - 77s 140ms/step - loss: 1.9283 - val_loss: 2.5814
Epoch 8/10
549/549 [==============================] - 76s 138ms/step - loss: 1.8248 - val_loss: 2.5633
Epoch 9/10
549/549 [==============================] - 76s 139ms/step - loss: 1.7274 - val_loss: 2.5610
Epoch 10/10
549/549 [==============================] - 76s 139ms/step - l

## 모델 평가하기

In [21]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=15):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
                                                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [33]:
print(generate_text(model, tokenizer, init_sentence="<start> i love", max_len=15))
print(generate_text(model, tokenizer, init_sentence="<start> i can", max_len=15))

<start> i love you , i m not gonna crack <end> 
<start> i can t help it if i wanted to <end> 


## validation loss 줄이기

validation loss를 줄이기 위해 hidden_size를 1024에서 2048로 증가시켰습니다.

In [58]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 2048
model_2 = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [59]:
for src_sample, tgt_sample in dataset.take(1): break
model_2(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 7.04482372e-05, -1.16639003e-05,  1.19799050e-04, ...,
         -5.14395797e-05, -1.00212543e-04,  2.33797095e-04],
        [ 5.09734891e-05,  7.97531175e-05, -4.19307726e-05, ...,
         -2.28043893e-04, -1.16066258e-04,  1.38302537e-04],
        [ 1.13307600e-04,  2.97581311e-04, -5.71191020e-04, ...,
         -3.43477092e-04, -3.10152740e-04,  1.06233128e-04],
        ...,
        [ 1.29651889e-05, -3.85121733e-04,  5.74893875e-07, ...,
          1.97815709e-04,  2.58338900e-04, -7.11387565e-06],
        [ 6.05874287e-04, -4.81581112e-04, -3.45603212e-06, ...,
          4.28660598e-04,  3.25734611e-04, -1.65696110e-04],
        [ 1.27169525e-03, -5.74430334e-04, -2.81298326e-05, ...,
          6.97627664e-04,  3.30417795e-04, -3.87523411e-04]],

       [[ 7.04482372e-05, -1.16639003e-05,  1.19799050e-04, ...,
         -5.14395797e-05, -1.00212543e-04,  2.33797095e-04],
        [-2.45659205e-04, -1.13071073e-05,  2

In [60]:
model_2.summary()

Model: "text_generator_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      multiple                  3072256   
_________________________________________________________________
lstm_14 (LSTM)               multiple                  18882560  
_________________________________________________________________
lstm_15 (LSTM)               multiple                  33562624  
_________________________________________________________________
dense_7 (Dense)              multiple                  24590049  
Total params: 80,107,489
Trainable params: 80,107,489
Non-trainable params: 0
_________________________________________________________________


In [62]:

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model_2.compile(loss=loss, optimizer=optimizer)
model_2.fit(dataset, epochs=10, validation_data=(enc_val, dec_val), verbose=1)

Epoch 1/10
549/549 [==============================] - 214s 390ms/step - loss: 3.5673 - val_loss: 3.1577
Epoch 2/10
549/549 [==============================] - 214s 390ms/step - loss: 2.9818 - val_loss: 2.8786
Epoch 3/10
549/549 [==============================] - 222s 405ms/step - loss: 2.6664 - val_loss: 2.6867
Epoch 4/10
549/549 [==============================] - 225s 410ms/step - loss: 2.3634 - val_loss: 2.5471
Epoch 5/10
549/549 [==============================] - 224s 408ms/step - loss: 2.0743 - val_loss: 2.4473
Epoch 6/10
549/549 [==============================] - 229s 418ms/step - loss: 1.8067 - val_loss: 2.3753
Epoch 7/10
549/549 [==============================] - 225s 410ms/step - loss: 1.5713 - val_loss: 2.3364
Epoch 8/10
549/549 [==============================] - 223s 405ms/step - loss: 1.3721 - val_loss: 2.3166
Epoch 9/10
549/549 [==============================] - 225s 411ms/step - loss: 1.2132 - val_loss: 2.3189
Epoch 10/10
549/549 [==============================] - 218s 397m

val_loss: 2.5487 -> 2.3349로 감소하였습니다.

In [64]:
def generate_text(model_2, tokenizer, init_sentence="<start>", max_len=15):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
                                                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [65]:
print(generate_text(model_2, tokenizer, init_sentence="<start> i love", max_len=15))
print(generate_text(model_2, tokenizer, init_sentence="<start> i can", max_len=15))

<start> i love forgiveness forgiveness forgiveness iv iv iv moves yin suicide suicide suicide babys 
<start> i can explicit tea street lamb prick acting caps caps cot cot quick cursive 


## 6. 루브릭

1. 가사 텍스트 생성 모델이 정상적으로 동작하는가?

hidden_size가 1024일 때 가사 텍스트 생성 모델이 정상적으로 작동했습니다.
hidden_size가 2048일 때는 가사 텍스트 생성 모델에서 단어가 비정상적으로 반복되는 모습을 보입니다.

2. 데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가?

특수문자 제거, 토크나이저 생성, 패딩처리 등의 과정이 빠짐없이 진행되었습니다.
토큰화 과정에서 토큰의 개수를 15개로 제한하였습니다.(maxlen = 15)

3. 텍스트 생성모델이 안정적으로 학습되었는가?

텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?

hidden_size를 1024에서 2048로 증가시켜서 validation loss를 줄이는데 성공하였지만 여전히 2.3349의 값으로 2.2 이하로 낮추는 데는 실패하였다. epoch 8회 이상부터 validation loss가 증가한 것을 보니 모델이 overfitting 되었음을 알 수 있습니다.

이 프로젝트에서는 validation loss를 줄이기 위해서 hidden_size만 2배 증가시켰는데 hyperparameter인 embedding_size 또한 증가시키면 validation loss가 감소될 것이라고 추측됩니다. 데이터가 크기 때문에 모델의 학습 시간이 오래걸렸는데 모델의 학습 시간을 줄이기 위한 공부가 필요할 것 같습니다.